<a href="https://colab.research.google.com/github/tnvsl/Projects/blob/main/Copy_of_Mag2021_OpenNMT_py_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Инструкция OpenNMT-py**

Автор: Елена Шукшина

**OpenNMT-py** – это открытая библиотека для нейронного машинного перевода, использующая PyTorch для реализации алгоритмов машинного обучения. Она проста в использовании и не требует глубоких знаний в области нейронных сетей.

В этом блокноте мы познакомимся с оснвными принциами работы библиотеки OpenNMT-py. В качестве примера мы обучим модель французско-английского перевода на небольшом параллельном корпусе (137 860 редложений) с очень маленьким словарем (всего 357 слов на французском и 231 слово на английском). 


**Перед началом удостоверьтесь, что в *Изменить > Настройки блокнота > Аппаратный ускоритель* выбран GPU.**

## **Шаг 1. Установка**

Скачать репозиторий OpenNMT-py версии 1.2.0 (запустите код, нажав на стрелочку в квадратных скобках ниже)

Первая строчка скачивает репозиторий, вторая – меняет версию на нужную. 

После выполнения ячейки в меню с папочкой слева можно будет посмотреть содержимое репозитория.

In [1]:
!git clone https://github.com/OpenNMT/OpenNMT-py
!cd OpenNMT-py && git checkout 60125c8

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 17644, done.
remote: Counting objects: 100% (603/603), done.
remote: Compressing objects: 100% (298/298), done.
remote: Total 17644 (delta 360), reused 485 (delta 295), pack-reused 17041
Receiving objects: 100% (17644/17644), 273.78 MiB | 28.41 MiB/s, done.
Resolving deltas: 100% (12668/12668), done.
Note: checking out '60125c8'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 60125c80 Bump v1.2.0 (#1850)


Установить OpenNMT-py

In [2]:
!python OpenNMT-py/setup.py install

running install
running bdist_egg
running egg_info
creating OpenNMT_py.egg-info
writing OpenNMT_py.egg-info/PKG-INFO
writing dependency_links to OpenNMT_py.egg-info/dependency_links.txt
writing entry points to OpenNMT_py.egg-info/entry_points.txt
writing requirements to OpenNMT_py.egg-info/requires.txt
writing top-level names to OpenNMT_py.egg-info/top_level.txt
writing manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
reading manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
writing manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying OpenNMT_py.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying OpenNMT_py.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying OpenNMT_py.egg-info/dependency_links.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying

Убедиться, что установлена правильная версия torchtext

In [3]:
!pip install torchtext==0.4.0

     |████████████████████████████████| 53 kB 1.4 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


##**Шаг 2. Загрузка данных**

Следующий блок скачивает файлы small_vocab_en и small_vocab_ru с https://github.com/susanli2016/NLP-with-Python/tree/master/data и разделяет данные на тренировочные, контрольные (5000 предложений) и тестовые (1000 предложений).

* Тренировочные данные (training data) используются для обучения модели.
* На контрольных данных (validation data) модель периодически проверяется в процессе обучения, чтобы избежать переобучения.
* Тестовые данные (test data) используются для оценки работы модели после завершения обучения.

In [4]:
# скачиваем репозиторий с данными
!git clone https://github.com/susanli2016/NLP-with-Python.git

Cloning into 'NLP-with-Python'...
remote: Enumerating objects: 324, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 324 (delta 1), reused 0 (delta 0), pack-reused 317
Receiving objects: 100% (324/324), 28.18 MiB | 11.17 MiB/s, done.
Resolving deltas: 100% (131/131), done.


Посмотрим число предложений в файлах:

In [5]:
# команда cd используется для смены рабочей папки
# команда wc в linux выводит на консоль число строк, слов и символов в файлах, указанных после нее
! cd NLP-with-Python/data && wc small_vocab_fr small_vocab_en

  137860  1961295 10135742 small_vocab_fr
  137860  1823250  9085267 small_vocab_en
  275720  3784545 19221009 total


Также посмотрим на их содержимое. Сравните английские и французские предложения.

In [6]:
# команда head в linux позволяет посмотреть первые 10 строк файла
# параметр -v добавляет название файла
# параметр -n позволяет менять число выводимых строк
!head -v -n 10 NLP-with-Python/data/small_vocab_fr
!head -v -n 10 NLP-with-Python/data/small_vocab_en      

==> NLP-with-Python/data/small_vocab_fr <==
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
california est généralement calme en mars , et il est généralement chaud en juin .
les états-unis est parfois légère en juin , et il fait froid en septembre .
votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .
son fruit préféré est l'orange , mais mon préféré est le raisin .
paris est relaxant en décembre , mais il est généralement froid en juillet .
new jersey est occupé au printemps , et il est jamais chaude en mars .
notre fruit est moins aimé le citron , mais mon moins aimé est le raisin .
les états-unis est parfois occupé en janvier , et il est parfois chaud en novembre .
==> NLP-with-Python/data/small_vocab_en <==
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is us

Как мы видим, эти данные уже токенизированы и приведены к нижнему регистру.

Теперь разделим их на тренировочные, контрольные и тестовые. Код очень простой -- просмотрите его.


In [7]:
# этот блок переменых испольщуется для называния файлов.
data = "NLP-with-Python/data/small_vocab"
path = "OpenNMT-py/data/"
val = path + "demo_val"
train = path + "demo_train"
test = path + "demo_test"
fr = "_fr.txt"
en =  "_en.txt"

def divide(datafile, valfile, trainfile, testfile):
    data = open(datafile, encoding = 'UTF-8')
    val = open(valfile, 'w', encoding = 'UTF-8')
    train = open(trainfile, 'w', encoding = 'UTF-8')
    test = open(testfile, 'w', encoding = 'UTF-8')

    # обратите внимание, в этой функции для print-а. Посмотрите, как они сработают ниже.
    print('Dividing', datafile)
    for i, line in enumerate(data):
        if i < 1000:
            test.write(line)
        elif (i >= 1000) and (i<6000) :
            val.write(line)
        else:
            train.write(line)
    print("Complete")

    data.close()
    val.close()
    train.close()
    test.close()

# предыдущая функция вызывается два раза
divide(data+"_fr", val+fr, train+fr, test+fr)
divide(data+"_en", val+en, train+en, test+en)

Dividing NLP-with-Python/data/small_vocab_fr
Complete
Dividing NLP-with-Python/data/small_vocab_en
Complete


##**Шаг 3. Предобработка данных**

Следующая команда запускает предварительную обработку данных, в ходе которой будут созданы файлы pyTorch с тренировочными и контрольными данными и словарем. 

In [8]:
!cd OpenNMT-py && python preprocess.py -train_src data/demo_train_fr.txt -train_tgt data/demo_train_en.txt -valid_src data/demo_val_fr.txt -valid_tgt data/demo_val_en.txt -save_data data/demo

[2021-11-05 17:07:46,147 INFO] Extracting features...
[2021-11-05 17:07:46,147 INFO]  * number of source features: 0.
[2021-11-05 17:07:46,147 INFO]  * number of target features: 0.
[2021-11-05 17:07:46,147 INFO] Building `Fields` object...
[2021-11-05 17:07:46,147 INFO] Building & saving training data...
[2021-11-05 17:07:46,380 INFO] Building shard 0.
[2021-11-05 17:07:51,514 INFO]  * saving 0th train data shard to data/demo.train.0.pt.
[2021-11-05 17:07:55,522 INFO]  * tgt vocab size: 231.
[2021-11-05 17:07:55,523 INFO]  * src vocab size: 357.
[2021-11-05 17:07:55,565 INFO] Building & saving validation data...
[2021-11-05 17:07:55,595 INFO] Building shard 0.
[2021-11-05 17:07:55,678 INFO]  * saving 0th valid data shard to data/demo.valid.0.pt.


Используемые параметры:
* -train_src тренировочные данные на входном языке
* -train_tgt тренировочные данные на выходном языке
* -valid_src контрольные данные на входном языке
* -valid_tgt контрольные данные на выходном языке
* -save_data путь для сохранения обработанных данных


## **Шаг 4. Обучение модели**

Следующая команда запускает обученние нейронной сети. 

На экране можно следить за значениями acc и ppl. Первое – точность (accuracy), она будет расти по мере обучения. Второе – перплексия (perplexity), чем она ниже, тем лучше. Также будут выводиться эти значения для контрольных данных.
Обучаться будет долго.


In [9]:
!cd OpenNMT-py && python train.py -data data/demo -save_model demo-model -train_steps 3000 -save_checkpoint_steps 500 -valid_steps 500 -world_size 1 -gpu_ranks 0

[2021-11-05 17:08:12,177 INFO]  * src vocab size = 357
[2021-11-05 17:08:12,177 INFO]  * tgt vocab size = 231
[2021-11-05 17:08:12,177 INFO] Building model...
[2021-11-05 17:08:26,849 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(357, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(231, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (linear

Используемые параметры:
* -data путь к обработанным данным
* -save_model путь для сохранения модели
* -train_steps время обучения – число шагов
* -save_checkpoint_steps как часто сохранять модель – число шагов
* -valid_steps как часто оценивать модель – число шагов
* -world_size число GPU
* -gpu_ranks ранги используемых GPU

По умолчанию используется модель с двумя слоями LSTM по 500 нейронов для декодера и энкодера.

## Шаг 5. Перевод

С помощью следующей команды можно перевести любой файл. Мы переведем тестовые данные, которые создали в начале.

In [10]:
#!cd OpenNMT-py && python translate.py -model demo-model_step_5000.pt -src data/demo_test_fr.txt -output pred.txt -replace_unk
!cd OpenNMT-py && python translate.py -model demo-model_step_3000.pt -src data/demo_test_fr.txt -output pred.txt -replace_unk

[2021-11-05 17:14:10,950 INFO] Translating shard 0.
/content/OpenNMT-py/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:23.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, u

Используемые параметры: 
* -model путь к файлу модели
* -src путь к файлу с тестовыми данными на входном языке
* -output путь для сохранения переводов модели
* -replace_unk  заменить токены UNK на токены входного языка с максимальным весом внимания

Также можно использовать:
* -verbose выводить на консоль переводы

Посмотрим на переводы

In [14]:
print('Оригинал:')
!head -n 15 OpenNMT-py/data/demo_test_fr.txt
print('-' * 50) #разделитель
print('Перевод модели:')
!head -n 15 OpenNMT-py/pred.txt
print('-' * 50) #разделитель
print('Эталонный перевод:')
!head  -n 15 OpenNMT-py/data/demo_test_en.txt

Оригинал:
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
california est généralement calme en mars , et il est généralement chaud en juin .
les états-unis est parfois légère en juin , et il fait froid en septembre .
votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .
son fruit préféré est l'orange , mais mon préféré est le raisin .
paris est relaxant en décembre , mais il est généralement froid en juillet .
new jersey est occupé au printemps , et il est jamais chaude en mars .
notre fruit est moins aimé le citron , mais mon moins aimé est le raisin .
les états-unis est parfois occupé en janvier , et il est parfois chaud en novembre .
la chaux est son moins aimé des fruits , mais la banane est mon moins aimé.
il a vu un vieux camion jaune .
inde est pluvieux en juin , et il est parfois chaud en novembre .
ce chat était mon animal le plus aimé .

# **ЗАДАЧКА!!!** Поменяйте, пожалуйста, код в предыдущем блоке так, чтобы видеть не 5 предложений, а 15.

## Шаг 6. Оценка модели

**BLEU** – автоматическая метрика оценки качества перевода, которая использует идею «чем ближе машинный перевод к переводу профессионального переводчика, тем он лучше». Алгоритм сравнивает n-граммы из перевода-кандидата с эталонным переводом, также производится подсчет совпадений.  Чем больше число совпадений, тем лучше качество перевода-кандидата. BLEU является, пожалуй, самой используемой метрикой оценки машинного перевода. Именно ее значения можно увидеть в научных статьях и таблицах результатов соревнований по машинному переводу. Например, лучший результат соревнований на тестовых данных WMT19 в русско-английском переводе составил 40.2, а в англо-русском – 36.3.

In [15]:
!cd OpenNMT-py && perl tools/multi-bleu.perl data/demo_test_en.txt < pred.txt

BLEU = 93.91, 97.4/95.0/92.8/90.5 (BP=1.000, ratio=1.000, hyp_len=13209, ref_len=13205)


Так как наш корпус имеет ограниченный словарь и тест проводился на части корпуса, BLEU окажется нереально высоким по сравнению с результатами соревнований. Если провести тест на независимых данных, результат будет совсем иной – большинство слов попросту не будут известны нашей модели.

# Шаг 7. Немножко теории, для просмотра -- выполнять не требуется

# Другие модели
Ниже список параметров, которые можно использовать для изменения архитектуры модели. В квадратных скобах перечислены возможные значения праметров.


-model_type [text|img|audio|vec] тип модели

-encoder_type [rnn|brnn|ggnn|mean|transformer|cnn] тип энкодера

-decoder_type [rnn|transformer|cnn] тип декодера

-layers число слоев в энкодере и декодере (или по отдельности -enc_layers и -dec_layers)

-rnn_size число скрытых состояний RNN (или по отдельности -enc_rnn_size и -dec_rnn_size)

-rnn_type [LSTM, GRU, SRU] тип RNN


Больше разных параметров обучения можно найти в официальной документации https://opennmt.net/OpenNMT-py/options/train.html


# А как же трансформер?

Возможно, вы уже слышали, что одной из самых передовых архитектур для машинного перевода сейчас является Трансформер, предложенный в научной статье Attention is all you need. Как можно его использовать?

Трансформер требует использования графических процессоров (GPU). Если у вас есть компьютер хотя бы с одной мощной видеокартой, можно попробовать запустить Трансформер с помощью вот такой команды.

python train.py -data data/demo -save_model model_demo -layers 6 -rnn_size 512 \
-word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer \
-decoder_type transformer -position_encoding -train_steps 80000 \
 -max_generator_batches 2 -dropout 0.1 -batch_size 2048 -batch_type tokens \
